# CS 5542 — Week 1 Lab
## From Data to Retrieval: GitHub → Colab → Hugging Face → Embeddings

**Learning Goals:**
- Use GitHub for collaborative analytics workflows
- Run notebooks in Google Colab
- Load datasets and models from Hugging Face Hub
- Build an embedding-based retrieval system (mini-RAG)


### GenAI Systems Context (Mini-RAG)
This lab implements a **mini Retrieval-Augmented Generation (RAG)** pipeline:
- A **Transformer encoder** produces semantic embeddings
- A **vector index (FAISS)** enables fast retrieval
- Retrieved context is what a downstream **LLM** would use for grounded generation


## Step 1 — Environment Setup
Install required libraries. This may take ~1 minute.


In [1]:
!pip install -q transformers datasets sentence-transformers faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 59.1 MB/s eta 0:00:00


## Step 2 — Load Dataset & Model from Hugging Face Hub
We use a lightweight news dataset and a sentence embedding model.


Replace you hugging face token in the empty string, on the mentioned commented line.

In [2]:
from huggingface_hub import login

HF_TOKEN = "..."  # <-- REPLACE THE EMPTY STRING WITH YOUR HF TOKEN

if HF_TOKEN and HF_TOKEN != "YOUR_HF_TOKEN_HERE":
    login(token=HF_TOKEN)
    print("✅ Logged in to Hugging Face")
else:
    print("⚠️ No HF token provided. Public models may still work, but rate limits may apply.")

✅ Logged in to Hugging Face


TASK

1. Find out any /ag_news dataset from the huggingface.
2. Look for "Use this dataset" button on the left side --> Use huggingface library option.
3. Copy the entire code and paste in the empty cell and run it successfully.

In [3]:
#paste your dataset code from huggingface here
from datasets import load_dataset
dataset = load_dataset("fancyzhx/ag_news")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/18.6M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

In [4]:
texts = dataset["train"].select(range(200))
print(f"Selected {len(texts)} examples from the 'train' split.")

Selected 200 examples from the 'train' split.


TASK

1. Find out the sentence-transformer -
all-MiniLM-L6-v2 from the huggingface website.
2. Look for "Use this model" button on the left side --> Use sentence-transformer library option.
3. Copy the first 2 lines of the code and paste in the empty cell and run it successfully.

In [12]:
#paste your first 2 lines of the sentence-transformer library code from the hugginface to load the model

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

## Step 3 — Create Embeddings
These vectors represent semantic meaning and enable retrieval before generation.


In [13]:
embeddings = model.encode(texts, show_progress_bar=True)
print('Embedding shape:', embeddings.shape)

Batches:   0%|          | 0/7 [00:00<?, ?it/s]

Embedding shape: (200, 384)


## Step 4 — Build a Vector Index (FAISS)
This simulates the retrieval layer in RAG systems.


In [14]:
import faiss
import numpy as np

dim = embeddings.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.array(embeddings))
print('Index size:', index.ntotal)

Index size: 200


## Step 5 — Retrieval Function
Search for documents related to a query.


In [15]:
def search(query, k=3):
    q_emb = model.encode([query])
    distances, indices = index.search(np.array(q_emb), k)
    return [texts[int(i)] for i in indices[0]]

## Step 6 — Try It!


In [17]:
query = "High oil prices"
top_chunks = search(query, k=3)

print(f"Top 3 chunks retrieved for query: '{query}'\n")
for i, chunk in enumerate(top_chunks):
    print(f"{i+1}. {chunk['text']}\n")

Top 3 chunks retrieved for query: 'High oil prices'

1. Oil prices soar to all-time record, posing new menace to US economy (AFP) AFP - Tearaway world oil prices, toppling records and straining wallets, present a new economic menace barely three months before the US presidential elections.

2. No Need for OPEC to Pump More-Iran Gov  TEHRAN (Reuters) - OPEC can do nothing to douse scorching  oil prices when markets are already oversupplied by 2.8 million  barrels per day (bpd) of crude, Iran's OPEC governor said  Saturday, warning that prices could fall sharply.

3. Oil and Economy Cloud Stocks' Outlook  NEW YORK (Reuters) - Soaring crude prices plus worries  about the economy and the outlook for earnings are expected to  hang over the stock market next week during the depth of the  summer doldrums.



## Reflection
In 1–2 sentences, explain how embeddings enable retrieval before generation in GenAI systems.


RAG allows the conversion of text into embeddings. The embeddings allow RAG systems to do a similarity search instead of a strict text search, or even a regex-based text search which can be useful when words are misspelled or not given in the exact order that is expected.